<a href="https://colab.research.google.com/github/rachel1234561/DS2002_Project/blob/main/converting_and_summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rachel1234561/DS2002_Project.git

Cloning into 'DS2002_Project'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 56 (delta 28), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 4.62 MiB | 2.94 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
#we need to figure out 3. Modify the number of columns from the source to the destination,
#reducing or adding columns. If you add data cols you can put any other useful
#information in that vcolumn you wish and also converting from json.

In [3]:
import pandas as pd
import json
import requests
import sqlite3

In [28]:
#the function takes the data being converted and the format of the data
def convert_data(input_data, input_format):
    # make input data a pandas df and handle errors
    if input_format == 'csv':
        #save csv as df
        df = pd.read_csv(input_data)
        #print a summary of the data file ingestion
        num_records = len(df)
        num_columns = len(df.columns)
        print("Summary of the data file ingestion:")
        print("Number of records:" + str(num_records))
        print("Number of columns:" + str(num_columns))
        print("")
    elif input_format == 'json':
        #save json as df
        data = json.loads(input_data)
        #code below resolves the issue of data dictionaries within data dictionaries
        columns_data = data['meta']['view']['columns']
        columns = [column['name'] for column in columns_data]
        df = pd.DataFrame(data['data'], columns=columns)
        #print a summary of the data file ingestion
        print("summary of the data file ingestion:\n")
        print("Number of records:" + str(len(df)))
        print("Number of columns:" + str(len(df.columns)))
    else:
        print("Invalid input format. Choose 'csv' or 'json'.")

    #ask use to input their prefered file type
    output_format = input('Enter output format (csv, json, sql): ')
    # Output data as the correct file type
    #saves a csv to files
    if output_format == 'csv':
        df.to_csv('output.csv')
        # Post-processing summary
        print("Post-processing summary:")
        print("Number of records:", len(pd.read_csv('output.csv')))
        print("Number of columns:", len(pd.read_csv('output.csv').columns))
    #saves a json file to files (also might need to debug this part)
    elif output_format == 'json':
        df.to_json('output.json')
        # Post-processing summary (I used stack overflow to debug this part)
        with open('output.json', 'r') as f:
         data = json.load(f)
        print("Post-processing summary:")
        print("Number of records:", len(df))
        print("Number of columns:", len(data))
    #saves to a sql db + db file in files
    elif output_format == 'sql':
      connection = sqlite3.connect('output.db')
      cursor = connection.cursor()
      connection.close()
      # Post-processing summary (SQL)
      connection = sqlite3.connect('output.db')
      cursor = connection.cursor()
      # ensure data type is compatable with sql
      for column in df.columns:
        if df[column].dtype not in [int, float, bool, object,str]:
            df[column] = df[column].astype(str)
        df[column] = df[column].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x) #I used ai debug to add this line
      df.to_sql('output', connection, if_exists='replace', index=False)
      cursor.execute("SELECT COUNT(*) FROM output")
      num_records = cursor.fetchone()[0]
      cursor.execute("SELECT COUNT(*) FROM pragma_table_info('output')")
      num_columns = cursor.fetchone()[0]
      connection.close()
      print("Post-processing summary (SQL):")
      print("Number of records:", num_records)
      print("Number of columns:", num_columns)
    else:
        print("Invalid output format. Choose 'csv', 'json', or 'sql'.")

In [5]:
#pass the path to the csv to the function
convert_data('/content/DS2002_Project/cereal.csv', 'csv')

Summary of the data file ingestion:
Number of records:77
Number of columns:16

Enter output format (csv, json, sql): sql
Post-processing summary (SQL):
Number of records: 77
Number of columns: 16


In [29]:
#pass json to the file
#this is an api from data.gov that has adoptoble pets from Montgomery County of Maryland?
url = 'https://data.montgomerycountymd.gov/api/views/e54u-qx42/rows.json?accessType=DOWNLOAD'
response = requests.get(url)
convert_data(response.text, 'json')

summary of the data file ingestion:

Number of records:66
Number of columns:25
Enter output format (csv, json, sql): sql
Post-processing summary (SQL):
Number of records: 66
Number of columns: 25


In [30]:
#this is to test to see if the SQL is working
connection = sqlite3.connect('output.db')
cursor = connection.cursor()
cursor.execute("SELECT * FROM output")
rows = cursor.fetchall()
for row in rows:
    print(row)
connection.close()

('row-4pk9-dhqa_2t62', '00000000-0000-0000-3530-4063B535CC5A', 0, 1728921638, None, 1728921643, None, '{ }', 'A523905', 'OWNER SUR', '2024-09-24T00:00:00', 'CHEECH', 'DOG', '4 YEARS', 'LARGE', 'WHITE', 'AM PIT BULL TER / MIX', 'N', '["http://www.petharbor.com/get_image.asp?res=DETAIL&id=A523905&location=MONT", null]', None, None, None, None, None, None)
('row-rjji-qjqv.qziq', '00000000-0000-0000-059A-B6F811A6CB79', 0, 1728921638, None, 1728921643, None, '{ }', 'A521383', 'FOSTER', '2024-08-29T00:00:00', '*WALDO', 'DOG', '1 YEAR 3 MONTHS', 'LARGE', 'BROWN / WHITE', 'AM PIT BULL TER', 'N', '["http://www.petharbor.com/get_image.asp?res=DETAIL&id=A521383&location=MONT", null]', None, None, None, None, None, None)
('row-b79i-kd2p-2267', '00000000-0000-0000-96E3-36D095BD066D', 0, 1714507237, None, 1714507244, None, '{ }', 'A518499', 'OWNER SUR', '2024-04-27T00:00:00', 'DJ', 'OTHER', 'NO AGE', 'SMALL', 'GREEN', 'TURTLE / REDEARED SLIDER', 'F', '["http://www.petharbor.com/get_image.asp?res=DET

In [31]:
#this is to test to see if the json part is working
#I used stack to figure out how to show the output.json in pretty print
with open('output.json', 'r') as f:
  data = json.load(f)
print(json.dumps(data, indent=1))

{
 "sid": {
  "0": "row-4pk9-dhqa_2t62",
  "1": "row-rjji-qjqv.qziq",
  "2": "row-b79i-kd2p-2267",
  "3": "row-w8mr-8cn5.wvxu",
  "4": "row-u9uv-mn9j-3sw2",
  "5": "row-ggrz-t92v.hmx5",
  "6": "row-ydq3.5gqi-jqax",
  "7": "row-x2hn-m58w.dm3d",
  "8": "row-34rj~3y9n~4n8k",
  "9": "row-84ri~qkd4-4tp2",
  "10": "row-k7dw-fmev_8f8f",
  "11": "row-p4yb.s4jc~2z7r",
  "12": "row-us53.fqm9.g3dq",
  "13": "row-qick_msuv_6nhx",
  "14": "row-xwmx.yyai~fa3d",
  "15": "row-7hwr.73bi.udy2",
  "16": "row-suud_7tjt_dcyk",
  "17": "row-kpsu-bp94-b9h8",
  "18": "row-qn6s-pe5p_jgyu",
  "19": "row-db64~q24e_fvhr",
  "20": "row-ytrj.cqv7.3wm3",
  "21": "row-uxe3~kyb9-54ut",
  "22": "row-tncf.6ium_k48c",
  "23": "row-42eq~ya7d.xg59",
  "24": "row-yybc~325i_bpqt",
  "25": "row-v5k2.trte~ijch",
  "26": "row-6zin~gwms-q5ag",
  "27": "row-s2er~vas5_pi7e",
  "28": "row-2s5m~empg-4fp6",
  "29": "row-t4ay~sy5w~5s2u",
  "30": "row-cqb2.hjmm_r2vn",
  "31": "row-t83z-cbw2-7yup",
  "32": "row-ejnr-f949~tvzn",
  "33": 